In [17]:
import azureml.core
print("SDK version:", azureml.core.VERSION)
from azureml.core import Workspace

subscription_id = "4a66f470-dd54-4c5e-bd19-8cb65a426003"
resource_group  = "AML_Playground"
workspace_name  = "Teams_ws"

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
    print('https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource' + ws.get_details()['id'])
except:
    print('Workspace not found')

SDK version: 1.0.33
Library configuration succeeded
https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/4a66f470-dd54-4c5e-bd19-8cb65a426003/resourceGroups/AML_Playground/providers/Microsoft.MachineLearningServices/workspaces/Teams_ws


In [18]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "p100cluster"


compute_target = ws.compute_targets[cluster_name]
print('Found existing compute target.')

Found existing compute target.


In [19]:
ds = ws.get_default_datastore()
#upload local model
model_path_on_datastore = 'Teams_slot_model' #cased model,vocab is too small? Do not have frequent word like common
ds_model = ds.path(model_path_on_datastore)
ds.upload(src_dir=r'D:\dl_repo\Tagging_data\bert-base-English-cased-pytorch',
          target_path= model_path_on_datastore,
          overwrite=False,
          show_progress=True)
print(ds_model.as_mount())
#upload unsupervised local model
model_path_on_datastore = 'Communication_slot_model_unsupervised' #cased model,vocab is too small? Do not have frequent word like common
ds_model_unsupervised = ds.path(model_path_on_datastore)
ds.upload(src_dir=r'D:\dl_repo\Data_model\Communication_unsupervised',
          target_path= model_path_on_datastore,
          overwrite=False,
          show_progress=True)
print(ds_model_unsupervised.as_mount())
#upload local data set
path_on_datastore = 'datasets/Teams_communication'
ds_data_communication = ds.path(path_on_datastore)
ds.upload(src_dir=r'D:\dl_repo\Data_model\Communication_data',
          target_path= path_on_datastore,
          overwrite=False,
          show_progress=True)
#upload pre-trained model
#TODO: reuse azure blob path in aml
model_path_on_datastore = 'Communication_slot_model_fine-tuned' #cased model,vocab is too small? Do not have frequent word like common
ds_model_pretrained = ds.path(model_path_on_datastore)
ds.upload(src_dir=r'D:\dl_repo\Data_model\Communication_finetuned',
          target_path= model_path_on_datastore,
          overwrite=False,
          show_progress=True)
print(ds_model.as_mount())


Target already exists. Skipping upload for Teams_slot_model\bert_config.json
Target already exists. Skipping upload for Teams_slot_model\pytorch_model.bin
Target already exists. Skipping upload for Teams_slot_model\vocab.txt
Target already exists. Skipping upload for Communication_slot_model_unsupervised\bert_config.json
Target already exists. Skipping upload for Communication_slot_model_unsupervised\pytorch_model.bin


$AZUREML_DATAREFERENCE_9933ac0ddaa9470193e54d7496725aa9


Target already exists. Skipping upload for Communication_slot_model_unsupervised\vocab.txt
Target already exists. Skipping upload for datasets/Teams_communication\BIO-tags.txt
Target already exists. Skipping upload for datasets/Teams_communication\comm_train_prod.txt


$AZUREML_DATAREFERENCE_62d59b6cc94c46a7bd07f9105f2d5e7c


Target already exists. Skipping upload for datasets/Teams_communication\Target_set_message_new_conll.txt
Target already exists. Skipping upload for datasets/Teams_communication\test.txt
Target already exists. Skipping upload for datasets/Teams_communication\test_blind_old.txt
Target already exists. Skipping upload for datasets/Teams_communication\test_nonormal.txt
Target already exists. Skipping upload for datasets/Teams_communication\train.txt
Target already exists. Skipping upload for datasets/Teams_communication\train_19k_only.txt
Target already exists. Skipping upload for datasets/Teams_communication\train_legacy_1m.txt
Target already exists. Skipping upload for datasets/Teams_communication\train_positive_generated.txt
Target already exists. Skipping upload for datasets/Teams_communication\train_teams.txt
Target already exists. Skipping upload for datasets/Teams_communication\train_valid_oct.txt
Target already exists. Skipping upload for datasets/Teams_communication\valid.txt
Targe

$AZUREML_DATAREFERENCE_b26a7627609c40049df3fe41c281e7b3


# Create an experiment


In [24]:
from azureml.core import Experiment

experiment_name = 'Similarity_search' 
experiment = Experiment(ws, name=experiment_name)

In [25]:
##BATCH AI
from azureml.train.dnn import PyTorch




script_params = {
    '--corpus_input':ds.path(f'azureml-blobstore-1dd9ddd7-6d52-40ad-8b00-be73cd90fd63/lm_data_civ.txt').as_mount(),
    '--query_input':'sample_data/teamsmessage_raw.txt'
}

estimator10 = PyTorch(source_directory='..', 
                    script_params=script_params,
                    compute_target=compute_target, 
                    entry_script='src/sentence_similarity_search.py',
                    #pip_packages=['pandas','pytorch-pretrained-bert==0.4.0','seqeval==0.0.5'],
                    #pip_packages=['pandas','pytorch-pretrained-bert==0.6.1','seqeval==0.0.5','nltk'],
                    pip_packages=['pandas','transformers','pytorch-pretrained-bert==0.6.1','seqeval==0.0.5','nltk','faiss-gpu','sentence-transformers'],
                    use_gpu=True)

In [26]:
print(estimator10.run_config.environment.docker.base_image)
print(estimator10.conda_dependencies.serialize_to_string())

mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - pandas
  - transformers
  - pytorch-pretrained-bert==0.6.1
  - seqeval==0.0.5
  - nltk
  - faiss-gpu
  - sentence-transformers
  - azureml-defaults
  - torch==1.0
  - torchvision==0.2.1
  - horovod==0.15.2



In [27]:
run = experiment.submit(estimator10)
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…